# 0. GAN
### reference 
- https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py
- https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice
- paper : https://arxiv.org/abs/1406.2661

# 1. Library Import

In [2]:
!pip install easydict

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6350 sha256=a1fc36f5f06448f8be14a872704b7bfd19a590807c6b70b4e69e985d5bbfa979
  Stored in directory: /root/.cache/pip/wheels/88/96/68/c2be18e7406804be2e593e1c37845f2dd20ac2ce1381ce40b0
Successfully built easydict


In [1]:
# import argparse # -> jupyter notebook X
import easydict

import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [10]:
# make images folder
os.makedirs('images', exist_ok = True)

# 2. Parameter Setting

In [30]:
opt = easydict.EasyDict({"n_epochs" : 5,
                         "batch_size" : 64,
                         "lr" : 0.0002,
                         "b1" : 0.5,
                         "b2" : 0.99,
                         "n_cpu" : 8,
                         "latent_dim" : 100,
                         "img_size" : 28,
                         "channels" : 1,
                         "sample_interval" : 400})

In [12]:
print(opt)

{'n_epochs': 200, 'batch_size': 64, 'lr': 0.0002, 'b1': 0.5, 'b2': 0.99, 'n_cpu': 8, 'latent_dim': 100, 'img_size': 28, 'channels': 1, 'sample_interval': 400}


In [13]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

In [14]:
print(img_shape)

(1, 28, 28)


In [15]:
cuda = True if torch.cuda.is_available() else False

In [16]:
print(cuda)

True


# 3. Model

In [17]:
# With Learnable Parameters
m = nn.BatchNorm1d(100)
# Without Learnable Parameters
m = nn.BatchNorm1d(100, affine=False)
input = torch.randn(20, 100)
output = m(input)

In [18]:
input.shape

torch.Size([20, 100])

In [19]:
output.shape

torch.Size([20, 100])

### 3.1 Generator

In [22]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        # Linear -> batchNorm (on, off) -> LeakyReLU 
        def block(in_feat, out_feat, normalize = True):
            layers = [nn.Linear(in_feat, out_feat)]
            # mini batch normalization
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            # slope : 0.2 -> if x is negative x * 0.2
            layers.append(nn.LeakyReLU(0.2, inplace = True))
            return layers
        
        # img_shape : 1 * 28 * 28, prod(img_shape) : 784
        # tanh : exp(x) - exp(-x) / exp(x) + exp(-x) , range : (-1, 1)
        self.model = nn.Sequential(*block(opt.latent_dim, 128, normalize = False),
                                   *block(128, 256),
                                   *block(256, 512),
                                   *block(512, 1024),
                                   nn.Linear(1024, int(np.prod(img_shape))),
                                   nn.Tanh()
                                  )
    # z is random noise
    # z size : batch * latent size
    def forward(self, z):
        img = self.model(z)
        # img size : batch * prod(img_shape)
        img = img.view(img.size(0), *img_shape)
        return img

### 3.2 Discriminator

In [23]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        # probability result
        self.model = nn.Sequential(nn.Linear(int(np.prod(img_shape)), 512),
                                   nn.LeakyReLU(0.2, inplace = True),
                                   nn.Linear(512, 256),
                                   nn.LeakyReLU(0.2, inplace = True),
                                   nn.Linear(256, 1),
                                   nn.Sigmoid()
                                  )
    
    # img size : batch size * channel * height * width
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

### 3.3 Loss Function

In [24]:
# Binary Cross Entropy Loss
adversarial_loss = torch.nn.BCELoss()

# 4. Data Loader and Model Training

In [25]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [26]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

### 4.1 Data Loader (MNIST)

In [27]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2021-04-22 13:48:39--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-04-22 13:48:40--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [             <=>    ]  33.20M  12.8MB/s    in 2.6s    

2021-04-22 13:48:43 (12.8 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [28]:
# data loader
os.makedirs("./", exist_ok = True)
dataloader = torch.utils.data.DataLoader(
                datasets.MNIST(
                    "./",
                    train = True,
                    download = True,
                    transform = transforms.Compose(
                        [transforms.Resize(opt.img_size), transforms.ToTensor(),
                         transforms.Normalize([0.5], [0.5])]
                    )
                ),
                batch_size = opt.batch_size,
                shuffle = True
            )

### 4.2 Model Training

In [29]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [31]:
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Adversarial ground truths
        # truths y value
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad = False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad = False)
        
        # Confiqure input
        real_imgs = Variable(imgs.type(Tensor))
        
        ###################
        # Train Generator #
        ###################
        optimizer_G.zero_grad()
        
        # sample noise as generator input
        # using the normal distribution
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))
        
        # Generate a batch of images
        gen_imgs = generator(z)
        
        # Loss measures generator's ability to fool discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        
        g_loss.backward()
        optimizer_G.step()
        
        #######################
        # Train Discriminator #
        #######################
        optimizer_D.zero_grad()
        
        # Measure discriminator's ability to classify real from generated samples
        # generate images detach
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()
        
        if i % 20 == 0:
            print("[Epoch %d / %d] [Batch %d / %d] [D loss: %f] [G loss: %f]"
                  % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
                 )
        
        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow = 5, normalize = True)

[Epoch 0 / 5] [Batch 0 / 938] [D loss: 0.708330] [G loss: 0.685343]
[Epoch 0 / 5] [Batch 20 / 938] [D loss: 0.416714] [G loss: 0.616385]
[Epoch 0 / 5] [Batch 40 / 938] [D loss: 0.492486] [G loss: 0.606595]
[Epoch 0 / 5] [Batch 60 / 938] [D loss: 0.622108] [G loss: 0.453586]
[Epoch 0 / 5] [Batch 80 / 938] [D loss: 0.458567] [G loss: 0.820725]
[Epoch 0 / 5] [Batch 100 / 938] [D loss: 0.479186] [G loss: 0.709180]
[Epoch 0 / 5] [Batch 120 / 938] [D loss: 0.455426] [G loss: 0.811841]
[Epoch 0 / 5] [Batch 140 / 938] [D loss: 0.377666] [G loss: 1.227942]
[Epoch 0 / 5] [Batch 160 / 938] [D loss: 0.463690] [G loss: 0.772292]
[Epoch 0 / 5] [Batch 180 / 938] [D loss: 0.705680] [G loss: 0.858457]
[Epoch 0 / 5] [Batch 200 / 938] [D loss: 0.613616] [G loss: 0.564179]
[Epoch 0 / 5] [Batch 220 / 938] [D loss: 0.513266] [G loss: 1.158929]
[Epoch 0 / 5] [Batch 240 / 938] [D loss: 0.675653] [G loss: 0.439165]
[Epoch 0 / 5] [Batch 260 / 938] [D loss: 0.516442] [G loss: 0.680341]
[Epoch 0 / 5] [Batch 280 /

In [33]:
z.shape

torch.Size([32, 100])

In [35]:
imgs.shape

torch.Size([32, 1, 28, 28])